# 自动微分

假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导

In [22]:
import torch

x = torch.arange(4.0)
# x= torch.arange(4.0,requires_grad=True)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度

In [23]:
x.requires_grad_(True)
x.grad

现在计算$y$

In [27]:
y =   2*torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度

In [28]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [29]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算`x`的另一个函数

In [37]:
print(x.grad)
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])


tensor([1., 1., 1., 1.])

深度学习中
，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和

In [40]:
x.grad.zero_()
y = x * x
y.sum().backward()#通常对标量进行求导
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [47]:
x.grad.zero_()
y = x * x
u = y.detach()#将y做成一个常数赋值给u,u作为常数，之前的的梯度记录也清零
z = u * x

z.sum().backward()
x.grad == u
print(x.grad)

tensor([0., 1., 4., 9.])


In [48]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x
print(x.grad)

tensor([0., 2., 4., 6.])


即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [50]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        print(b.norm())
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
print('a:',a)
d = f(a)
d.backward()

a.grad == d / a

a: tensor(1.3182, requires_grad=True)
tensor(2.6364, grad_fn=<LinalgVectorNormBackward0>)
tensor(5.2729, grad_fn=<LinalgVectorNormBackward0>)
tensor(10.5457, grad_fn=<LinalgVectorNormBackward0>)
tensor(21.0915, grad_fn=<LinalgVectorNormBackward0>)
tensor(42.1829, grad_fn=<LinalgVectorNormBackward0>)
tensor(84.3658, grad_fn=<LinalgVectorNormBackward0>)
tensor(168.7316, grad_fn=<LinalgVectorNormBackward0>)
tensor(337.4633, grad_fn=<LinalgVectorNormBackward0>)
tensor(674.9266, grad_fn=<LinalgVectorNormBackward0>)


tensor(True)